In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [3]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False
)

In [5]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [6]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [7]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [8]:
sample = source.get_batch(num_seq_per_batch=1024)

In [9]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [10]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [{'DivideBy': {'divisor': 986.98462}}],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 607.1142}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [11]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [12]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [13]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm_experiments')
db = client.neuralnilm_experiments

In [14]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    db=db,
    experiment_id=0,
    metrics=Metrics([4]),
    learning_rate=1E-2,
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02


In [15]:
trainer.fit(400)

INFO:neuralnilm.trainer:Starting training for 400 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 400 iterations.


In [16]:
net.train_iterations

400

In [17]:
import pymongo
list(db.train_scores.find(
        filter={'experiment_id': 0},
        sort=[('iteration', pymongo.ASCENDING)],
        projection=['iteration', 'loss']
    ).limit(50))

[{u'_id': ObjectId('55dc7f9c42c364177d7dca43'),
  u'iteration': 0,
  u'loss': 1.6276246617549675},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca47'),
  u'iteration': 1,
  u'loss': 1.5228960085254801},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca48'),
  u'iteration': 2,
  u'loss': 1.5860146993464577},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca49'),
  u'iteration': 3,
  u'loss': 1.4338984674466222},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca4a'),
  u'iteration': 4,
  u'loss': 1.5653352890982746},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca4b'),
  u'iteration': 5,
  u'loss': 1.2985009795571358},
 {u'_id': ObjectId('55dc7f9c42c364177d7dca4c'),
  u'iteration': 6,
  u'loss': 1.2646140071012004},
 {u'_id': ObjectId('55dc7f9d42c364177d7dca4d'),
  u'iteration': 7,
  u'loss': 1.245193783712254},
 {u'_id': ObjectId('55dc7f9d42c364177d7dca4e'),
  u'iteration': 8,
  u'loss': 1.167900972993815},
 {u'_id': ObjectId('55dc7f9d42c364177d7dca4f'),
  u'iteration': 9,
  u'loss': 1.2287597227552924},
 {u'_id': Ob

In [18]:
list(db.validation_scores.find({'fold': 'train', 'metric_type': 'classification'}).sort('iteration', pymongo.ASCENDING).limit(50))

[]

In [19]:
agg_pipeline = [
    {"$group": {
            "_id": "$experiment_id",
            "max_acc": {"$max": "$SyntheticAggregateSource.unseen_appliances.2_state_classification.accuracy_score"},
            "max_iteration": {"$max": "$iteration"}
        }}
]
list(db.validation_scores.aggregate(agg_pipeline))

[{u'_id': 0, u'max_acc': None, u'max_iteration': 400}]

In [20]:
from monary import Monary

In [21]:
monary = Monary()

In [22]:
iterations, loss, source_id = monary.query(
    db='neuralnilm_experiments',
    coll='train_scores',
    query={
        'experiment_id': 0,
        'iteration': {'$gt': 0}
    },
    fields=['iteration', 'loss', 'source_id'],
    types=['int32', 'float32', 'int8']
)

In [23]:
import pandas as pd

df = pd.DataFrame({'loss': loss, 'source_id': source_id}, index=iterations)

In [24]:
import matplotlib.pyplot as plt
plt.plot(df.index, df['loss'].values)
plt.show()

In [27]:
from neuralnilm.monitor.monitor import Monitor

mon = Monitor(0)

In [28]:
mon._plot_train_scores()

In [75]:
mon._plot_validation_scores()

In [30]:
mon.validation_metric_names

[u'regression.mean_squared_error',
 u'regression.relative_error_in_total_energy',
 u'regression.mean_absolute_error',
 u'classification_2_state.f1_score',
 u'classification_2_state.recall_score',
 u'classification_2_state.accuracy_score',
 u'classification_2_state.precision_score']

In [33]:
df.plot?